In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('House Price Prediction Dataset.csv')

In [3]:
df.head()

,Id,Area,Bedrooms,Bathrooms,Floors,YearBuilt,Location,Condition,Garage,Price
0,1,1360,5,4,3,1970,Downtown,Excellent,No,149919
1,2,4272,5,4,3,1958,Downtown,Excellent,No,424998
2,3,3592,2,2,3,1938,Downtown,Good,No,266746
3,4,966,4,2,2,1902,Suburban,Fair,Yes,244020
4,5,4926,1,4,2,1975,Downtown,Fair,Yes,636056


In [28]:
df.shape

(2000, 10)

In [10]:
df.sample(5)

,Id,Area,Bedrooms,Bathrooms,Floors,YearBuilt,Location,Condition,Garage,Price
949,950,3951,2,1,1,1939,Rural,Poor,Yes,360625
112,113,3657,5,1,2,1980,Rural,Excellent,No,902987
1194,1195,3901,1,4,1,1968,Urban,Excellent,Yes,134335
472,473,2886,3,4,2,1923,Rural,Excellent,Yes,493333
1715,1716,944,3,2,2,1932,Suburban,Fair,Yes,176951


In [4]:
df.describe()

,Id,Area,Bedrooms,Bathrooms,Floors,YearBuilt,Price
count,2000.000000,2000.000000,2000.000000,2000.00000,2000.000000,2000.000000,2000.000000
mean,1000.500000,2786.209500,3.003500,2.55250,1.993500,1961.446000,537676.855000
std,577.494589,1295.146799,1.424606,1.10899,0.809188,35.926695,276428.845719
min,1.000000,501.000000,1.000000,1.00000,1.000000,1900.000000,50005.000000
25%,500.750000,1653.000000,2.000000,2.00000,1.000000,1930.000000,300098.000000
50%,1000.500000,2833.000000,3.000000,3.00000,2.000000,1961.000000,539254.000000
75%,1500.250000,3887.500000,4.000000,4.00000,3.000000,1993.000000,780086.000000
max,2000.000000,4999.000000,5.000000,4.00000,3.000000,2023.000000,999656.000000


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Id         2000 non-null   int64 
 1   Area       2000 non-null   int64 
 2   Bedrooms   2000 non-null   int64 
 3   Bathrooms  2000 non-null   int64 
 4   Floors     2000 non-null   int64 
 5   YearBuilt  2000 non-null   int64 
 6   Location   2000 non-null   object
 7   Condition  2000 non-null   object
 8   Garage     2000 non-null   object
 9   Price      2000 non-null   int64 
dtypes: int64(7), object(3)
memory usage: 156.4+ KB


In [6]:
df.isnull().sum()

,0
Id,0
Area,0
Bedrooms,0
Bathrooms,0
Floors,0
YearBuilt,0
Location,0
Condition,0
Garage,0
Price,0


In [8]:
df['Location'].value_counts() # we can go for ordinal encoding downtown > urban > suburban > rural

,count
Location,
Downtown,558
Urban,485
Suburban,483
Rural,474


In [11]:
df['Condition'].value_counts()  # we will apply ordinal encoding here also order : Excellent > gppd > fair > poor

,count
Condition,
Fair,521
Excellent,511
Poor,507
Good,461


In [13]:
df['Garage'].value_counts()  # we can apply one hot encoding here

,count
Garage,
No,1038
Yes,962


In [25]:
X = df.drop(columns=['Id', 'Price'], axis=1)
y = df['Price']

In [27]:
X.shape, y.shape

((2000, 8), (2000,))

In [31]:
X.sample()

,Area,Bedrooms,Bathrooms,Floors,YearBuilt,Location,Condition,Garage
359,3459,2,3,2,2005,Urban,Poor,Yes


In [29]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [14]:
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [32]:
transform1 = ColumnTransformer(transformers=[
    ('trf1', OneHotEncoder(drop='first'), [5]),
    ('trf2', OrdinalEncoder(categories=[['Poor', 'Fair', 'Good', 'Excellent']]), [6]),
    ('trf3', OneHotEncoder(drop='first'), [7]),
    ('trf4', StandardScaler(), [0,1,2,3,4])
])

In [33]:
transform2 = ColumnTransformer(transformers=[
    ('trf1', OneHotEncoder(drop='first'), [5]),
    ('trf2', OrdinalEncoder(categories=[['Poor', 'Fair', 'Good', 'Excellent']]), [6]),
    ('trf3', OneHotEncoder(drop='first'), [7]),
    # ('trf4', StandardScaler(), [0,1,2,3,4])
])

In [34]:
X_train_transformed = transform1.fit_transform(X_train)
X_test_transformed = transform1.transform(X_test)

In [35]:
X_train_scaled = transform2.fit_transform(X_train)
X_test_scaled = transform2.transform(X_test)

In [37]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [38]:
lr = LinearRegression()

lr.fit(X_train_transformed, y_train)

y_pred = lr.predict(X_test_transformed)

r2_score(y_test, y_pred)

-0.00814309997438234

In [39]:
from sklearn.ensemble import RandomForestRegressor

In [40]:
rfc = RandomForestRegressor(n_estimators=100)

rfc.fit(X_train_scaled, y_train)

RandomForestRegressor()

In [42]:
y_pred2 = rfc.predict(X_test_scaled)

In [45]:
r2_score(y_test, y_pred2)

-0.0245912585595649